This requires a dir of the csvs generated by pLannotate from the Addgene plasmids.

In [12]:
import pandas as pd
import glob
from Bio.Seq import Seq
import numpy as np
import plotly.express as px

df_list = []
path = "../tacc_gen_data/2022-08-03_csvs/*.csv"
for file in glob.glob(path):
    addgene_id = file.split("/")[-1].split(".")[0]
    df = pd.read_csv(file)
    df["addgene_id"] = addgene_id
    df_list.append(df)

df_list = pd.concat(df_list)

In [13]:
ori = df_list[df_list["sseqid"] == "ori"]
ori = ori[ori['fragment'] == False]
len(ori['sequence'].value_counts())

354

In [15]:
print(len(set(df_list['addgene_id'])))
print(len(df_list))

51384
983436


In [16]:
#df_list.to_csv("../out_data/2022-08-03_annotated_addgene.csv", index = None)

In [1]:
df_list = pd.read_csv("../out_data/2022-08-03_annotated_addgene.csv")
len(df_list) #1001099

983436

In [ ]:
# this is the main filtering

# general filtering for mostly full sequences
df_list_full = df_list.loc[df_list['fragment'] == False]
df_list_full = df_list_full.loc[df_list_full['percent identity'] > 95]
df_list_full = df_list_full.loc[df_list_full['percent match length'] > 95]

# total occurances of each gene/feature
# if there is not more than 1 obs, we drop it
df_list_full['total_sseqid_occurrences'] = df_list_full.groupby('sseqid')['sseqid'].transform('size')
# df_list_full = df_list_full.loc[df_list_full['total_sseqid_occurrences'] > 1]

# gives each unique sequence a subgroup number
# subgroup_id is basically a hash/map of the sequence
# counts the number of obs in each subgroup
# if there is not more than 1 obs, we drop it
df_list_full['subgroup_id'] = df_list_full.groupby(['sequence']).ngroup()
df_list_full['num_of_occurances_in_subgroup'] = df_list_full.groupby('subgroup_id')['subgroup_id'].transform('size')
# df_list_full = df_list_full.loc[df_list_full['num_of_occurances_in_subgroup'] > 1]

# adds most common variant info to each gene/feature
most_common_subgroups_representative_index = df_list_full.groupby("sseqid")['num_of_occurances_in_subgroup'].nlargest(n=1).index
most_common_subgroups_representative_index = [_[1] for _ in most_common_subgroups_representative_index]
most_common_subgroups = df_list_full.loc[most_common_subgroups_representative_index]['subgroup_id'].values
# df_list_full = df_list_full.loc[~df_list_full['subgroup_id'].isin(most_common_subgroups)]
# most_common_undocumented = df_list_full.loc[df_list_full['subgroup_id'].isin(most_common_subgroups)]
df_list_full['is_most_common'] = df_list_full['subgroup_id'].isin(most_common_subgroups)

# tallies the number of subgroups for each gene/feature
def unique(inGroup):
    return len(set(inGroup))
df_list_full['num_of_unique_subgroups'] = df_list_full.groupby('sseqid')['subgroup_id'].transform(unique)

# infernal has a wonky naming convention for sseqid, so we cant use it
# it seems to be removed with the above steps, but this is just in case
df_list_full = df_list_full.loc[df_list_full['database'] != "Rfam"]

# there is an NaN obs -- should figure out what it is
df_list_full = df_list_full.dropna()

# add PI info
pis = pd.read_csv("../out_data/addgene_id_pi.csv")
df_list_full = df_list_full.merge(pis, on="addgene_id", how="inner")

def unique(inGroup):
    return len(set(inGroup))
df_list_full['num_of_unique_pis'] = df_list_full.groupby('subgroup_id')['pi'].transform(unique)

# some hits have incorrect % identity and % match length
# this takes the max found among the subgroup and gives it a _max column
max_perc_iden_len = pd.DataFrame((df_list_full
                                    .groupby('subgroup_id')
                                    .apply(lambda _: 
                                        (max(_["subgroup_id"]),
                                         max(_['percent identity']),
                                         max(_['percent match length'])
                                        ))
                                    .to_list()
                                 ),
                                    columns=['subgroup_id','percent identity','percent match length']
                                )  
df_list_full = df_list_full.merge(max_perc_iden_len, on='subgroup_id', how='left', suffixes=['',' updated'])
df_list_full["has_updated_pi-pm"] = (
                                    (df_list_full['percent match length'] != df_list_full['percent match length updated']) | 
                                    (df_list_full['percent identity'] != df_list_full['percent identity updated'])
                                    )

# this works with the above code -- adds new database column which corrects issues subgroups being found by different means
# takes the database with the highest percent identity and match length
def get_fixed_database(row):
    if row['has_updated_pi-pm'] == False:
        return row['database']
    else:
        subgroup = row['subgroup_id']
        #uses global df_list_full which is not great, but easiest way to do this
        x = df_list_full.loc[df_list_full['subgroup_id'] == subgroup,:]
        updated_db = x.loc[(x['percent identity'] == x['percent identity updated']) & (x['percent identity'] == x['percent identity updated']),:].iloc[0]['database']
        x["updated_db"] = updated_db
        return updated_db
    
df_list_full["updated_database"] = df_list_full.apply(get_fixed_database, axis=1)

# add snapgene info
from Bio import SeqIO
snapgene = list(SeqIO.parse("../in_data/snapgene.fasta", "fasta"))
snapgene = pd.DataFrame([( _.id, str(_.seq), "snapgene") for _ in snapgene], columns=['sseqid','canon_sequence','database'])
df_list_full = df_list_full.merge(snapgene, on=["sseqid","database"], how="left")

# removes "cannonical" variants from each gene/feature
df_list_full['is_canon_seq'] = (df_list_full['percent identity updated'] == 100) & (df_list_full['percent match length updated'] == 100)

# add silent mutation info
def is_silent_mutation(row):
    if row["is_canon_seq"] == True:
        return True
    # if it's FPbase or SwissProt, and it's not 100% identical, it must not be a silent mutation
    elif (row['database'] != "snapgene") and (row['Type'] == "CDS"):
        return False
    # if it's not a CDS, it cannot be a silent mutation by definition
    elif row['Type'] != "CDS":
        return False
    # translate to test if it's a silent mutation (only for snapgene)
    elif (row['database'] == "snapgene") and (row['Type'] == "CDS"):
        canon_aa = Seq(row["canon_sequence"].replace("-","")).translate()
        found_aa = Seq(row["sequence"].replace("-","")).translate()
        return canon_aa == found_aa
    else:
        return np.nan
df_list_full["is_silent_mutation"] = df_list_full.apply(is_silent_mutation, axis=1)


print(len(df_list_full)) # prev_len: 59838
df_list_full.head()

In [5]:
#df_list_full.to_csv("../out_data/2022-08-03_full_plasmid_features_w_subgroups.csv", index=None)

In [1]:
import pandas as pd
df_list_full = pd.read_csv("../out_data/2022-08-03_full_plasmid_features_w_subgroups.csv")

trimmed_df = df_list_full.loc[df_list_full['total_sseqid_occurrences'] > 1]
trimmed_df = trimmed_df.loc[trimmed_df['num_of_occurances_in_subgroup'] > 1]
trimmed_df = trimmed_df.loc[trimmed_df['num_of_unique_pis'] > 1,:]

trimmed_df = trimmed_df[trimmed_df['is_canon_seq'] == False]

#trimmed_df = trimmed_df.loc[trimmed_df['updated_database'] != 'swissprot']

print(len(trimmed_df)) # prev_len: 59838

150914


In [5]:
print(f"tot num of non-canon seq: {len(df_list_full[df_list_full['is_canon_seq'] == False])}")

tot num of non-canon seq: 171828


In [48]:
#trimmed_df.to_csv("../out_data/2022-08-03_full_plasmid_features_w_subgroups_FILTERED.csv", index=None)

---

In [94]:
plotting = df_list_full.loc[df_list_full['is_canon_seq'] == False,:]
plotting = plotting.loc[plotting['Type'] != 'gene',:]

convert = {
 'oriT':'rep_origin', 
 'misc_RNA':'ncRNA', 
 'repeat_region':"misc_feature", 
 'misc_recomb':"misc_feature", 
 'tRNA':'ncRNA', 
 "5'UTR":"misc_feature", 
 "3'UTR":"misc_feature", 
 'sig_peptide':"misc_feature",
 }
plotting['Type'] = plotting['Type'].replace(convert)
plotting.loc[plotting['sseqid'].str.contains("WPRE"), 'Type'] = 'enhancer'
plotting.loc[plotting['sseqid'].str.contains("CEN___ARS"), 'Type'] = 'rep_origin'
plotting.loc[plotting['sseqid'].str.contains("IRES"), 'Type'] = 'IRES'
#plotting.loc[plotting['sseqid'].str.contains("MCS"), 'Type'] = 'MCS'
plotting.loc[plotting['sseqid'].str.contains("HIV_1_Psi"), 'Type'] = 'ncRNA'

plotting['Type'] = plotting['Type'].replace({"misc_feature":"other"})

plotting = plotting[['Type']].value_counts().reset_index()
plotting = plotting.rename(columns={0:'num. observations'})

plotting['Type'] = plotting['Type'].replace({"rep_origin":"origin of rep."})
plotting['Type'] = plotting['Type'].str.replace("_"," ")

col_order = plotting['Type'].to_list() # for next plot



plotting_2 = df_list_full.loc[df_list_full['is_canon_seq'] == False,:]
plotting_2 = plotting_2[['Type','sseqid','subgroup_id']].drop_duplicates()
plotting_2 = plotting_2.loc[plotting_2['Type'] != 'gene',:]

plotting_2['Type'] = plotting_2['Type'].replace(convert)
plotting_2.loc[plotting_2['sseqid'].str.contains("WPRE"), 'Type'] = 'enhancer'
plotting_2.loc[plotting_2['sseqid'].str.contains("CEN___ARS"), 'Type'] = 'rep_origin'
plotting_2.loc[plotting_2['sseqid'].str.contains("IRES"), 'Type'] = 'IRES'
#plotting_2.loc[plotting_2['sseqid'].str.contains("MCS"), 'Type'] = 'MCS'
plotting_2.loc[plotting_2['sseqid'].str.contains("HIV_1_Psi"), 'Type'] = 'ncRNA'

plotting_2['Type'] = plotting_2['Type'].replace({"misc_feature":"other"})



plotting_2 = plotting_2[['Type']].value_counts().reset_index()
plotting_2 = plotting_2.rename(columns={0:'unique observations'})

plotting_2['Type'] = plotting_2['Type'].replace({"rep_origin":"origin of rep."})
plotting_2['Type'] = plotting_2['Type'].str.replace("_"," ")

plotting_2 = plotting.merge(plotting_2, on='Type')
plotting_2 = plotting_2.melt(id_vars=['Type'], value_vars=['num. observations','unique observations'])
plotting_2['variable'] = plotting_2['variable'].replace('num. observations', 'total observations')

fig = px.scatter(plotting_2,
           x='value',
           y='Type',
           log_x=True,
           color='variable',
           # size ='tot_subgroup_by_type',
           template="simple_white",
           height=500,
           width=525,
           labels={'variable':''},
           # title="Total",
           
)
fig.update_layout(yaxis={'categoryorder':'total ascending'}) # add only this line
fig.update_yaxes(showgrid=True)
fig.update_xaxes(showgrid=True)
fig.update_traces(marker=dict(size=10,opacity=0.4))
fig.update_layout(showlegend=False)

#fig.write_image("../out_data/figures/total_variant_combined.svg")


In [96]:
x = df_list_full.loc[df_list_full['is_canon_seq'] == False,:].copy(deep = True)
x = x[['Type','sseqid','subgroup_id','percent identity updated', 'percent match length updated', 'full length of feature in db']].drop_duplicates()
x['num diff'] = round(x['full length of feature in db'] - (((x['percent identity updated']/100)) * x['full length of feature in db']))
x['num diff'] = x['num diff'].replace(0,1)
x['percent deviated'] = x['num diff'] / x['full length of feature in db'] * 100
x[['Type','sseqid','subgroup_id','percent deviated']]


plotting_3 = x[['Type','sseqid','subgroup_id','percent deviated']].drop_duplicates()
#plotting_3 = plotting_3[['Type','sseqid','num_of_unique_subgroups','subgroup_id']].drop_duplicates()

plotting_3 = plotting_3.loc[plotting_3['Type'] != 'gene',:]

convert = {
 'oriT':'rep_origin', 
 'misc_RNA':'ncRNA', 
 'repeat_region':"misc_feature", 
 'misc_recomb':"misc_feature", 
 'tRNA':'ncRNA', 
 "5'UTR":"misc_feature", 
 "3'UTR":"misc_feature", 
 'sig_peptide':"misc_feature",
 }
plotting_3['Type'] = plotting_3['Type'].replace(convert)
plotting_3.loc[plotting_3['sseqid'].str.contains("WPRE"), 'Type'] = 'enhancer'
plotting_3.loc[plotting_3['sseqid'].str.contains("CEN___ARS"), 'Type'] = 'rep_origin'
plotting_3.loc[plotting_3['sseqid'].str.contains("IRES"), 'Type'] = 'IRES'
#plotting_3.loc[plotting_3['sseqid'].str.contains("MCS"), 'Type'] = 'MCS'
plotting_3.loc[plotting_3['sseqid'].str.contains("HIV_1_Psi"), 'Type'] = 'ncRNA'

plotting_3['Type'] = plotting_3['Type'].replace({"misc_feature":"other"})



# plotting_3 = plotting_3[['Type']].value_counts().reset_index()
# plotting_3 = plotting_3.rename(columns={0:'unique observations'})

plotting_3['Type'] = plotting_3['Type'].replace({"rep_origin":"origin of rep."})
plotting_3['Type'] = plotting_3['Type'].str.replace("_"," ")


fig = px.box(plotting_3,
       x='percent deviated',
       y='Type',
       template="simple_white",
       range_x=[0,5.5],
       height=500,
       width=600,
)

fig.update_yaxes(categoryorder='array', categoryarray=col_order[::-1])
fig.update_traces(marker=dict(size=2,opacity=0.2))
fig.update_layout(yaxis={'side': 'right'})

# fig.write_image("../out_data/figures/total_variant_percent_deviated.svg")
fig


In [6]:
import numpy as np
import plotly.express as px

plot_data = df_list_full.sort_values(by = ['num_of_occurances_in_subgroup'],ascending = False)
cols = ['sseqid','subgroup_id','is_canon_seq','num_of_unique_pis','is_silent_mutation','num_of_unique_subgroups','total_sseqid_occurrences','num_of_occurances_in_subgroup','updated_database']
plot_data = plot_data[cols].drop_duplicates()

plot_data['subgroup_id'] = plot_data['subgroup_id'].astype(str)
plot_data['sseqid'] = plot_data['sseqid'].astype(str)
plot_data['is_silent_mutation'] = plot_data['is_silent_mutation'].fillna('idk')


#plot_data = df_list_full.sort_values(by = ['num_of_unique_pis'],ascending = False)
plot_data['num_of_unique_pis_log'] = np.log(plot_data['num_of_unique_pis'])
plot_data['low_pi'] = plot_data['num_of_unique_pis'] <= 2

plot_data = plot_data.query("is_canon_seq == False")

fig = px.scatter(plot_data,
                 x='num_of_occurances_in_subgroup', 
                 y='num_of_unique_pis',
                 log_y=True,
                 log_x=True,
                 template='simple_white',
                 #facet_col='is_canon_seq',
                 hover_data=cols,
                 height=500,
                 width=600,
                 labels={'num_of_occurances_in_subgroup':'Number of occurrences of variant',
                         'num_of_unique_pis':'Number of unique depositing labs'},
                 color = 'low_pi',
                 color_discrete_sequence=['#000000','red'],
                 opacity=0.3,
                 render_mode = 'svg'
                 #color = 'is_silent_mutation',
                 #color = 'updated_database',
                 #size = 'num_of_unique_subgroups'
                 )
#fig.update_xaxes(type='category')
#fig.update_layout(xaxis_categoryorder = 'total descending')
fig.add_hline(y=20, line_color='blue', line_width=2, col=1, opacity=0.5)
fig.add_vline(x=1205, line_color='orange', line_width=2, col=1, opacity=0.5)

fig.update_layout(showlegend=False)
fig.update_xaxes(mirror=True)
fig.update_yaxes(mirror=True)

#fig.write_image("../out_data/figures/fig2_b.svg")

fig


In [122]:
low_pi_tally = pd.DataFrame(plot_data['low_pi'].value_counts()).reset_index()
low_pi_tally = low_pi_tally.replace({True:'', #≤ 2 depositing labs
                      False:' '}) #> 2 depositing labs
low_pi_tally.columns = ['low_pi','count']
low_pi_tally = low_pi_tally.sort_values(by = ['count'],ascending = True)

fig = px.bar(low_pi_tally,
        x='low_pi',
        y='count',
        color_discrete_sequence=['#000000','red'],
        color='low_pi',
        template='plotly_white',
        labels={'low_pi':'',
                'count':'Number of unique variants'},
        height=500,
        width=400,
        )
fig.update_layout(showlegend=False)
fig.update_layout(xaxis_tickformat = ',')
fig.write_image("../out_data/figures/fig2_c.svg")

fig
